# Example PrivacyFingerprint Workflow

This notebook walks you through a potential end-to-end workflow, to introduce a user to how each component can be loaded and how they can be configured

In [ ]:
import os
import sys
from spacy import displacy

path_root = os.path.dirname(os.getcwd())

if path_root not in sys.path:
    sys.path.append(path_root)

In [ ]:
from src.generate.synthea import GenerateSynthea
from src.generate.llm import GenerateLLM
from src.extraction.extraction import Extraction
from src.standardise_extraction.standardise_extraction import (
    StandardiseExtraction,
)
from src.privacy_risk_scorer.privacy_risk_scorer import PrivacyRiskScorer
from src.privacy_risk_explainer.privacy_risk_explainer import (
    PrivacyRiskExplainer,
)

from src.config.experimental_config import load_experimental_config
from src.config.global_config import load_global_config

### Importing and Loading Global and Experimental Config

**global_config_path**  this is the location of the global config path and then the output folder name is redefined to ensure the example experiments are out in the open. (Normally the default output folder should be used for your own experiments.)

**default_config_path** is given so the user can point to the default experimental config values. Currently the pipeline copies the original experimental config down into the folder, and if this exists, only uses the experimental config defined in that folder.

In [ ]:
# Loads global config and redefines the outputs to go to an example_output (normally default to an outputs folder which is git ignored in this repo.)
global_config_path = "../config/global_config.yaml"
global_config = load_global_config(global_config_path)
global_config.output_paths.output_folder = "../example_output"


default_config_path = "../config/experimental_config.yaml"
experimental_config = load_experimental_config(default_config_path)
experimental_config.outputs.experiment_name = "example_pipeline_05_08_24"
experimental_folder = f"{global_config.output_paths.output_folder}/{experimental_config.outputs.experiment_name}"

# Privacy Fingerprint End-to-End Overview

The Pipeline has been broken down into four components:
1. **GenerateSynthea**: This generates a list of dictionary of synthetic patient records.
2. **GenerateLLM**: This generates medical notes using the outputs created from **GenerateSynthea**.
3. **Extraction**: This currently uses an LLM that is specialised to extract given entities from the synthetic medical notes produced by **GenerativeLLM**
4. **StandardiseExtraction**: This standardises the results extracted from the medical text.
5. **PrivacyRiskScorer**: This scores the uniqueness of standardised entity values extracted.
6. **PrivacyRiskExplainer**: Takes in the predicted transformed values, and transformed dataset generater from the gaussian copula, and calculates shapley values. 

Additionally each class will also take a path for the input required to create their output. This allows the user to break-up the pipeline and run from specific points in the pipeline.

## 1. GenerateSynthea: Generating Synthetic Patient Data using Synthea 

Synthea-international is an expansion of Synthea, which is an open-source synthetic patient generator that produces de-identified health records for synthetic patients.

GenerateSynthea is a class used to run Synthea. You will need to follow the instructions on the README to ensure Synthea is installed.

In [ ]:
experimental_config.synthea.path_output = f"{experimental_folder}/synthea.json"
experimental_config.synthea.population_num = "100"

In [ ]:
output_synthea = GenerateSynthea(
    global_config=global_config, syntheaconfig=experimental_config.synthea
).run_or_load()
output_synthea

## 2. GenerateLLM: Generating Synthetic Patient Medical Notes 

This component uses a LLM model either hosted via Ollama to generate synthetic patient medical notes depending on the prompt template given below.

### Create Prompt Templates used in the Experimental Pipeline.

This defines the template you want the generate component to use. In this example we use Llama2, and this is a prompt template that can be used to support this.

In [ ]:
# Functions needed to create prompt templates and save them for the experiments.
from src.config.prompt_template_handler import (
    save_generate_template_to_json,
    load_and_validate_generate_prompt_template,
)

# Defines the path of where llama3 template lives in the generate folder.
# A llama2 template can also be used. However, you will need to change the experimental config.
generate_template_path = (
    f"{global_config.output_paths.generate_template}llama3_template.json"
)

# This defines a template used by LLama3
generate_template = """<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a medical student answering an exam question about writing clinical notes for patients.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Keep in mind that your answer will be accessed based on incorporating all the provided information and the quality of prose.

1. Use prose to write an example clinical note for this patient's doctor.
2. Use less than three sentences.
3. Do not provide recommendations.
4. Use the following information:

{data}
<|eot_id|>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Saves the template to the path defined.
save_generate_template_to_json(
    template_str=generate_template, file_path=generate_template_path
)

# Loads the template so the user can inspect the template saved.
loaded_generate_template = load_and_validate_generate_prompt_template(
    filename=generate_template_path
)
print(loaded_generate_template)

This runs GenerateLLM using the synthea output from the previous run, and saves the LLM output to the given path_output_llm.

You can set **verbose** to true or false depending on whether you want outputs to print to the screen on run. 

In [ ]:
experimental_config.generate.synthea_path = (
    experimental_config.synthea.path_output
)
experimental_config.generate.path_output = f"{experimental_folder}/llm.json"

In [ ]:
output_llm = GenerateLLM(
    global_config=global_config, generateconfig=experimental_config.generate
).run_or_load()
output_llm

## 3. Extraction: Re-extracting Entities from the Patient Medical Notes

This uses a package called GliNER to extract out entities from the synthetic medical notes.

Changing inputs to the **experimental_config.extraction.entity_list** allows you to look for more entities

In [ ]:
experimental_config.extraction.llm_path = (
    experimental_config.generate.path_output
)
experimental_config.extraction.path_output = (
    f"{experimental_folder}/extraction.json"
)
experimental_config.extraction.entity_list = [
    "nhs number",
    "person",
    "date of birth",
    "diagnosis",
]
experimental_config.extraction.server_model_type = "gliner"

In [ ]:
output_extraction = Extraction(
    global_config=global_config,
    extractionconfig=experimental_config.extraction,
).run_or_load()
output_extraction

### Visualising Entities using DisplaCy

This visualises the entities in an example clinical note using DisplaCy.

We format the extracted entities into a dictionary compatable with DisplaCy, and display the string.

In [ ]:
string_id = 1

ents_dict = {
    "text": output_llm[string_id],
    "ents": output_extraction[string_id]["Entities"],
}

displacy.render(ents_dict, manual=True, style="ent")

## 4. StandardiseExtraction: Normalising Entities Extracted for Scoring

This takes in the above List of Dictionary entities and begins to normalise the responses into a dataframe format.

The standardisation process is broken down into many parts:
1. Entities are extracted from the object created from **Extraction**, and a set of functions can be applied to clean them during this process.
2. This creates a list of cleaned entities. Multiple entities can be extracted from the same person for a given entity type, for example diagnosis. Currently the codebase only takes the first entity given.
3. Next the outputs are normalised i.e. Dates can be written in multiple formats but have the same meaning.
4. Lastly the data is encoded and formatted as a numpy array ready for PyCorrectMatch

In the src/config.py file:

extra_preprocess_functions_per_entity defines how entities are cleaned while extracted from the extraction_output.

```
extra_preprocess_functions_per_entity = {"person": [clean_name.remove_titles]}
```

standardise_functions_per_entity defines how entities are extracted, and defines any normalisation process you may want on a column of entities.
```
standardise_functions_per_entity = {
    "person": [extract_first_entity_from_list],
    "nhs number": [extract_first_entity_from_list],
    "date of birth": [
        extract_first_entity_from_list,
        normalise_columns.normalise_date_column,
    ],
    "diagnosis": [extract_first_entity_from_list],
}
```

This uses the output_extraction value created by the **Extraction** class and saves the outputs of the normalisation process as a .csv to the given path.

In [ ]:
path_output_standardisation = f"{experimental_folder}/standardisation.csv"

In [ ]:
output_standards = StandardiseExtraction(
    extraction_input=output_extraction,
    path_output=path_output_standardisation,
    save_output=True,
).run()
output_standards

This loads an extraction input from the extraction_path provided, and creates the output_standards.

In [ ]:
output_standards = StandardiseExtraction(
    extraction_path=experimental_config.extraction.path_output,
    path_output=path_output_standardisation,
    save_output=False,
).run()
output_standards

This loads a pre-saved output_standards from the given path provided.

In [ ]:
output_standards = StandardiseExtraction(
    path_output=path_output_standardisation
).load()
output_standards

## 5. PrivacyRiskScorer: This scores the uniqueness of standardised entity values extracted.

In [ ]:
try:
    scorer = PrivacyRiskScorer(
        scorer_config=experimental_config.pycorrectmatch
    )
except ValueError as e:
    print(e)

In [ ]:
# Here we fit the model, this has to happen first before calculating scores or transforming
scorer.fit(output_standards)
# This is the transformed dataset from the real record values to the marginal values
transformed_dataset = scorer.map_records_to_copula(output_standards)
N_FEATURES = output_standards.shape[1]

## 6. PrivacyRiskExplainer: Takes in the predicted transformed values, and transformed dataset generater from the gaussian copula, and calculates shapley values. 

In [ ]:
# SHAP takes a while to run - a progress bar appears when running SHAP
try:
    explainer = PrivacyRiskExplainer(
        scorer.predict_transformed,
        N_FEATURES,
        explainer_config=experimental_config.explainers,
    )
except ValueError as e:
    print(e)

# Calculating shapley values using the transformed_dataset
local_shapley_df, global_shap, exp_obj = explainer.explain(transformed_dataset)

In [ ]:
global_shap

In [ ]:
# Plot the mean shap values - global explanation
explainer.plot_global_explanation(exp_obj)

In [ ]:
# Plot the local shap values for a particular record
explainer.plot_local_explanation(exp_obj, 49)

## 7. PyCanon 

Pycanon asseses the values of common privacy measuring metrics, such as K-Anonymity, I-Diversity and t-Closeness. 

For more information on these metrics see `docs/pycanon/pycanon_and_privacy_metrics.md`

Each entity used in extraction should be added to `config/experimental_config.yaml` under `pycanon` in order to be used for the following analysis. 

In [ ]:
identifiers = experimental_config.pycanon.identifiers
quasi_identifiers = experimental_config.pycanon.quasi_identifiers
sensitive_attributes = experimental_config.pycanon.sensitive_attributes

In [ ]:
from pycanon.anonymity import k_anonymity, t_closeness, l_diversity

print("K-Anonymity: ", k_anonymity(output_standards, quasi_identifiers))
print(
    "t-Closeness: ",
    t_closeness(output_standards, quasi_identifiers, sensitive_attributes),
)
print(
    "l-Diversity: ",
    l_diversity(output_standards, quasi_identifiers, sensitive_attributes),
)